In [3]:
from glob import glob
import numpy as np
import pandas as pd
import csv
import math
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier



In [67]:
preP = "oldCSVs/trainData-badChan/"
files = glob(preP+'*.csv')
print(len(files))
trainCsv = pd.concat( [ pd.read_csv(f) for f in files ] )

#combined_csv = trainCsv

combined_csv = trainCsv.query("label in ['absz', 'tcsz', 'tnsz', 'gnsz', 'cpsz', 'fnsz', 'spsz']")
combined_csv.to_csv("data-train-90.csv", index=False )


7


In [68]:
preP2 = "oldCSVs/testData-badChan/"
files2 = glob(preP2+'*.csv')
print(len(files2))
testCsv = pd.concat( [ pd.read_csv(f) for f in files2] )

#combined_csv2 = testCsv
combined_csv2 = testCsv.query("label in ['absz', 'tcsz', 'tnsz', 'gnsz', 'cpsz', 'fnsz', 'spsz']")
combined_csv2.to_csv("data-test-90.csv", index=False )




7


In [4]:
preP2 = "bestData/"
files2 = glob(preP2+'*/*.csv')
print(len(files2))
testCsv = pd.concat( [ pd.read_csv(f) for f in files2] )

#combined_csv2 = testCsv
combined_csv2 = testCsv.query("label in ['tnsz', 'gnsz', 'fnsz', 'spsz']")
combined_csv2.to_csv("all4.csv", index=False )




18


In [4]:
combined_csv.to_csv("trainNom.csv", index=False )
combined_csv2.to_csv("testNom.csv", index=False )

In [69]:
dataset = np.array(combined_csv.values.tolist())
labels = dataset[:,-1]
xTrain = dataset[:,:-1]

Ls = list(set(labels))
mappedLabels = [Ls.index(i) for i in list(labels)]

In [6]:
combined_csv2.shape

(623, 1920)

In [70]:
testset = np.array(combined_csv2.values.tolist())
testLabels = testset[:,-1]
xTest = testset[:,:-1]
mappedTestLabels = [Ls.index(i) for i in list(testLabels)]


In [8]:
countsTrain = [mappedLabels.count(i) for i in list(range(len(Ls)))]
countsTest = [mappedTestLabels.count(i) for i in list(range(len(Ls)))]
#countPredicted = [list(predicted).count(i) for i in list(range(len(Ls)))]


In [71]:
trainScaler = MinMaxScaler(feature_range=(-1, 1))
trainScaler.fit(xTrain)
xTrain = trainScaler.transform(xTrain)
finalTestX = trainScaler.transform(xTest)

X_train = xTrain
Y_train = mappedLabels

X_test = finalTestX
Y_test = mappedTestLabels

In [9]:
Ls

['gnsz', 'tcsz', 'spsz', 'cpsz', 'fnsz', 'tnsz', 'absz']

In [10]:
np.sum(countsTrain)

837

In [11]:
countsTrain

[210, 25, 41, 135, 359, 17, 50]

In [12]:
np.sum(countsTest)

623

In [13]:
countPredicted

NameError: name 'countPredicted' is not defined

In [7]:
testset.shape

(658, 1920)

In [14]:

def get_my_best_estimator(X, y, folds):

    params_grid = [{
        'C': [2**i for i in range(-5, 3)],
        'gamma': [2**i for i in range(-6, 2)]}]
    gs = GridSearchCV(SVC(), params_grid, n_jobs=2, cv=folds)
    gs.fit(X, y)
    best_estimator = gs.best_estimator_
    return best_estimator



trainScaler = MinMaxScaler(feature_range=(-1, 1))
trainScaler.fit(xTrain)
xTrain = trainScaler.transform(xTrain)
finalTestX = trainScaler.transform(xTest)

X_train = xTrain
Y_train = mappedLabels

X_test = finalTestX
Y_test = mappedTestLabels



bestEstimator = get_my_best_estimator(X_train, Y_train, 5)
bestC = bestEstimator.C#2.0 #bestEstimator.C
bestGamma = bestEstimator.gamma #0.03125 #bestEstimator.gamma

modelRbf = SVC(C=bestC , gamma=bestGamma, kernel='rbf', verbose=True, class_weight='balanced').fit(X_train, Y_train)


[LibSVM]

In [79]:
[2**i for i in range(-12, 4)]

[0.000244140625,
 0.00048828125,
 0.0009765625,
 0.001953125,
 0.00390625,
 0.0078125,
 0.015625,
 0.03125,
 0.0625,
 0.125,
 0.25,
 0.5,
 1,
 2,
 4,
 8]

In [1]:
modelRbf = SVC(C=0.00244140625,
               kernel='linear',
               decision_function_shape="ovr",
                class_weight='balanced',
               verbose=True).fit(X_train, Y_train)

#predicted = modelRbf.predict(finalTestX)
#f1_score(mappedTestLabels, predicted, average="weighted")

scores = cross_val_score(modelForest, X_train, Y_train, cv=5)
np.mean(scores)

NameError: name 'SVC' is not defined

In [74]:
f1_score(mappedTestLabels, predicted, average="weighted")

0.522307076926858

In [51]:
modelRbf = SVC(C=11,
               kernel='poly',
               decision_function_shape="ovr",
               verbose=True).fit(X_train, Y_train)

predicted = modelRbf.predict(finalTestX)
f1_score(mappedTestLabels, predicted, average="micro")

[LibSVM]

0.6243980738362761

In [26]:
scores = cross_val_score(modelRbf, X_train, Y_train, cv=5)
np.mean(scores)

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

0.7109780439121757

In [23]:
predicted = modelRbf.predict(finalTestX)
f1_score(mappedTestLabels, predicted, average="weighted")
#accuracy_score(mappedTestLabels,predicted )

0.5613930332498083

In [17]:
DTC = DecisionTreeClassifier()

adaDTC = AdaBoostClassifier(DTC, random_state=7)

ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "algorithm" : ["SAMME","SAMME.R"],
              "n_estimators" :[1,2],
              "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

gsadaDTC = GridSearchCV(adaDTC,param_grid = ada_param_grid, cv=10, scoring="f1_weighted", n_jobs=-1, verbose = 1)

gsadaDTC.fit(X_train,Y_train)

ada_best = gsadaDTC.best_estimator_

Fitting 10 folds for each of 112 candidates, totalling 1120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 1120 out of 1120 | elapsed: 10.7min finished


In [18]:
modelForest = ada_best.fit(X_train,Y_train)

scores = cross_val_score(modelForest, X_train, Y_train, cv=10)
np.mean(scores)

predicted = modelForest.predict(finalTestX)
f1_score(mappedTestLabels, predicted, average="weighted")

0.4705171518958581

In [67]:
fImp = list(ada_best.feature_importances_)

In [68]:
featuresInd = np.argsort(fImp)

In [69]:
features= np.sort(fImp)

In [71]:
features[1864:]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 2.82039500e-18, 4.93569126e-18,
       5.64079001e-18, 5.64079001e-18, 7.75608626e-18, 3.17548247e-03,
       3.17548247e-03, 3.17548247e-03, 3.70472955e-03, 3.94092730e-03,
       4.13316766e-03, 4.15409028e-03, 4.19927190e-03, 4.20497679e-03,
       6.53498583e-03, 6.77436260e-03, 7.23555445e-03, 7.40945910e-03,
       8.17251193e-03, 8.59703791e-03, 9.98008777e-03, 1.00909776e-02,
       1.01615439e-02, 1.06963620e-02, 1.07947139e-02, 1.15472090e-02,
       1.25642184e-02, 1.27411334e-02, 1.38007476e-02, 1.48189182e-02,
       1.50139040e-02, 1.79815428e-02, 1.87277806e-02, 2.49131685e-02,
       3.17969575e-02, 3.34828192e-02, 3.49292174e-02, 3.79365793e-02,
       3.89063939e-02, 4.68861312e-02, 4.98277733e-02, 5.01247195e-02,
       6.21543851e-02, 1.19486100e-01, 2.22049093e-01])

In [56]:
impF = featuresInd[1874:]

In [58]:
np.max(impF)

1915

In [59]:
impF

array([1500, 1269,  246,  140,  423,  849,  654, 1032,  873, 1233, 1808,
       1702, 1761, 1517, 1719,  369,   90, 1675, 1816, 1240, 1527, 1867,
       1304, 1723, 1135,  943,   94,  448, 1590, 1333, 1915,   46,  470,
       1572,  656,  712, 1440,  966, 1857, 1854,  525, 1753, 1724, 1396,
       1456])

In [ ]:
dataset = np.array(combined_csv.values.tolist())
labels = dataset[:,-1]
xTrain = dataset[:,:-1]

Ls = list(set(labels))
mappedLabels = [Ls.index(i) for i in list(labels)]

In [69]:
dataset.shape

(658, 1920)

In [61]:
bestTrain = combined_csv.iloc[:,list(impF)+[1919]]
bestTest = combined_csv2.iloc[:,list(impF)+[1919]]

In [62]:
bestTrain

,fft_std_30-60Hz_T6,fft_ptp_8-12Hz_T3,fft_energy_18-24Hz_T6,fft_boundedvariation_60-90Hz_FP1,fft_entropy_30-60Hz_F7,fft_median_18-24Hz_P4,fft_mean_8-12Hz_FP2,fft_min_30-60Hz_F8,fft_median_24-30Hz_T6,fft_ptp_1-4Hz_T5,...,fft_std_12-18Hz_T3,fft_min_8-12Hz_T4,time_nonlinenergy_PZ,time_nonlinenergy_O2,fft_max_12-18Hz_P3,time_max_F7,time_fractaldim_PZ,fft_std_4-8Hz_FZ,fft_std_18-24Hz_P3,label
0,110.038351,2337.612664,5.349235e+05,8.637547,-7.262107e+07,791.159856,3083.456937,67.184933,316.579417,9739.807456,...,561.057420,541.369149,539.199937,419.069166,3047.561061,302.235577,1.248265,5598.659455,427.785982,absz
1,129.454993,2986.852023,6.617390e+05,8.119807,-1.209839e+08,819.779504,4729.075337,20.514574,287.548800,8717.347184,...,748.661018,591.408319,705.317428,546.995675,3077.296844,254.457231,1.362856,5346.742763,464.400458,absz
2,129.465008,2661.424113,9.974846e+05,9.546365,-9.981389e+07,845.536461,3514.132552,72.380508,294.373116,11856.395026,...,615.721468,525.590413,586.937937,541.459340,2625.085746,235.250058,1.313786,3599.283487,414.106401,absz
3,158.101692,2255.845088,1.539570e+06,8.531317,-1.203307e+08,1123.132611,5661.177234,28.753059,361.529836,8740.635365,...,869.944993,418.923620,1606.243130,1989.245101,4962.193551,289.004419,1.355841,5331.876739,635.495292,absz
4,151.237459,3039.426057,1.379146e+06,9.269653,-1.249359e+08,1391.190480,5206.514706,61.143013,349.504314,8885.465547,...,987.384502,457.964697,1620.363676,1804.078269,4539.825409,280.709465,1.369368,4657.375614,588.474430,absz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,24.655353,759.703480,3.408605e+04,8.925087,-1.892839e+06,144.459941,507.001346,10.323467,52.970514,2151.845021,...,121.101126,119.812028,29.158889,23.106346,675.202532,52.987041,1.535718,776.998179,105.740226,cpsz
229,23.135281,860.432207,3.091248e+04,8.917918,-2.248918e+06,145.997280,545.821967,8.798377,46.920243,1783.823197,...,136.640033,89.929126,29.515237,23.935456,687.584199,56.284304,1.538552,842.372795,96.542610,cpsz
230,23.350507,753.450985,3.583379e+04,9.299823,-1.361344e+06,162.571697,540.809174,10.145743,47.177228,1848.670836,...,137.643436,101.433808,30.990496,25.028742,655.229249,45.017005,1.563848,660.762800,112.817761,cpsz
231,23.050229,686.812568,2.783798e+04,8.701862,-1.768292e+06,146.669663,509.190121,8.859476,52.441745,2200.179896,...,119.666873,113.106454,26.066615,19.589126,613.349206,48.154839,1.524002,826.145813,102.859281,cpsz


In [63]:
bestTrain.to_csv("trainBest.csv", index=False )
bestTest.to_csv("testBest.csv", index=False )

In [45]:
bestTrain

,fft_std_30-60Hz_T5,fft_ptp_8-12Hz_PZ,fft_energy_18-24Hz_T5,fft_boundedvariation_60-90Hz_F8,fft_entropy_30-60Hz_F4,fft_median_18-24Hz_P3,fft_mean_8-12Hz_FP1,fft_min_30-60Hz_F7,fft_median_24-30Hz_T5,fft_ptp_1-4Hz_T4,...,fft_mean_24-30Hz_FP2,fft_std_12-18Hz_PZ,fft_min_8-12Hz_T3,time_nonlinenergy_P4,time_nonlinenergy_O1,fft_max_12-18Hz_O2,time_max_F4,time_fractaldim_P4,fft_std_4-8Hz_FP2,fft_std_18-24Hz_O2
0,95.038542,3931.268165,5.871824e+05,9.191066,-1.381069e+08,948.013000,3258.051623,31.539694,270.910388,7950.722306,...,447.262252,808.321777,676.013265,364.532073,326.335846,2833.066703,331.277857,1.260064,4397.365769,400.647772
1,116.176116,3841.073168,8.738140e+05,6.388093,-1.015778e+08,967.825113,4350.105624,29.203035,294.523668,7007.502286,...,647.426758,737.806855,674.026998,409.132965,446.379336,2723.736331,473.834805,1.384696,3626.857016,438.249173
2,124.037128,4081.800868,8.576543e+05,8.024720,-1.384424e+08,821.784154,3723.466113,29.751495,316.850431,7791.467241,...,597.094980,913.454526,709.746331,507.884071,387.530320,3350.146831,285.682428,1.309867,3666.578566,461.350998
3,101.210637,7380.116297,1.276793e+06,7.979562,-2.502178e+08,909.063153,6046.319680,39.733420,244.134506,4088.148002,...,813.327592,1756.804356,632.001570,1021.617321,1559.977300,6417.198344,447.056025,1.379998,2613.849724,1078.277163
4,120.995472,6599.958570,9.654468e+05,9.544372,-3.358430e+08,996.522673,5607.099601,31.941579,295.552722,4633.682448,...,793.160100,1799.193248,452.689840,1525.317589,1299.355499,6307.856085,412.906651,1.382264,2533.732231,1116.171331
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,24.418414,1051.401385,3.199377e+04,9.627960,-2.201467e+06,152.247528,506.755113,11.507142,57.298668,1742.581682,...,78.176168,206.724029,114.116202,22.516956,28.743178,646.468487,66.837027,1.572988,767.700757,113.850963
229,21.806137,1069.479779,2.215345e+04,9.318139,-1.924272e+06,152.283263,569.127280,15.108795,42.920785,1621.193948,...,87.322048,199.994960,108.836701,21.288220,25.323213,653.102548,69.032890,1.563709,867.284528,112.400901
230,22.985457,973.737829,3.577024e+04,9.743408,-1.968802e+06,166.125125,541.727006,8.123622,50.613931,1731.909071,...,84.230423,200.520151,131.025462,22.646958,28.408664,698.676981,61.797132,1.557272,668.681402,122.028482
231,23.343950,1029.697974,2.815721e+04,9.561957,-1.780340e+06,154.527189,515.834318,11.232257,46.789214,2117.909832,...,82.078371,179.351046,110.841112,19.496585,25.749074,595.239874,58.802636,1.554403,812.869906,99.248131
